## 🗂️ Descripción de Entidades y Campos Clave
1. Descripción de Entidades y Campos Clave

A. Entidad: DEPARTAMENTO (Dimensión)
Esta tabla actúa como un catálogo maestro. Contiene la información única de cada departamento geográfico.

Llave Primaria (PK): id_departamento (Integer). Identificador único. No puede ser nulo.

Campos:

nombre: Nombre del departamento (e.g., "Antioquia").

codigo_dane: Código estándar de geolocalización (usualmente string para preservar ceros a la izquierda).

B. Entidad: INCIDENTE (Hechos/Transaccional)
Esta tabla almacena los eventos o sucesos reportados.

Llave Primaria (PK): id_incidente (Integer). Identificador único del reporte.

Llave Foránea (FK): id_departamento. Conecta el incidente con la tabla DEPARTAMENTO. Relación 1:N (Un departamento tiene muchos incidentes).

Campos de Análisis:

municipio: Ubicación específica.

fecha_hecho: Cuándo ocurrió el evento (Date).

genero, grupo_etario: Datos demográficos.

armas_medios: Modalidad del incidente.

cantidad: Métrica numérica (BigInt) para agregaciones (sumas, promedios).

## Propuesta de Implementación 

In [0]:
%sql
-- Tabla Dimensional: DEPARTAMENTO
CREATE TABLE IF NOT EXISTS departamento (
    id_departamento INT COMMENT 'Llave primaria del departamento',
    nombre STRING COMMENT 'Nombre del departamento',
    codigo_dane STRING COMMENT 'Código estandarizado DANE'
) USING PARQUET;

-- Tabla Transaccional: INCIDENTE
CREATE TABLE IF NOT EXISTS incidente (
    id_incidente INT COMMENT 'Llave primaria del incidente',
    id_departamento INT COMMENT 'FK hacia la tabla departamento',
    municipio STRING,
    fecha_hecho DATE,
    genero STRING,
    grupo_etario STRING,
    armas_medios STRING,
    cantidad BIGINT COMMENT 'Métrica de conteo'
) USING PARQUET
-- Recomendación: Particionar por fecha si el volumen de datos es alto
PARTITIONED BY (fecha_hecho);

## 📖 Diccionario de Datos

### 1. Entidad: `DEPARTAMENTO`
Información maestra de las divisiones geográficas.

| Campo | Tipo (SQL) | Tipo (Spark) | Llave | Descripción |
| :--- | :--- | :--- | :---: | :--- |
| `id_departamento` | `INTEGER` | `IntegerType` | **PK** | Identificador único del departamento. |
| `nombre` | `VARCHAR(100)` | `StringType` | | Nombre descriptivo (ej. Antioquia). |
| `codigo_dane` | `VARCHAR(10)` | `StringType` | | Código oficial DANE (ej. "05"). |

### 2. Entidad: `INCIDENTE`
Registro transaccional de los hechos reportados.

| Campo | Tipo (SQL) | Tipo (Spark) | Llave | Descripción |
| :--- | :--- | :--- | :---: | :--- |
| `id_incidente` | `INTEGER` | `IntegerType` | **PK** | Identificador único del reporte. |
| `id_departamento` | `INTEGER` | `IntegerType` | **FK** | Relación con la tabla DEPARTAMENTO. |
| `municipio` | `VARCHAR(100)` | `StringType` | | Ciudad o municipio del hecho. |
| `fecha_hecho` | `DATE` | `DateType` | | Fecha del suceso (AAAA-MM-DD). |
| `genero` | `VARCHAR(20)` | `StringType` | | Género de la víctima/actor. |
| `grupo_etario` | `VARCHAR(30)` | `StringType` | | Rango de edad (ej. Adulto). |
| `armas_medios` | `VARCHAR(100)` | `StringType` | | Medio utilizado en el hecho. |
| `cantidad` | `BIGINT` | `LongType` | | Conteo numérico para agregaciones. |

---
**Glosario:**
* **PK:** Primary Key (Llave Primaria)
* **FK:** Foreign Key (Llave Foránea)

##Diagrama simple

%md
![](Diagrama.drawio.png)

## Versión de Databricks Runtime, tipo de clúster, núcleos/RAM, autoscaling.

In [0]:
import os
print("DATABRICKS_RUNTIME_VERSION:",os.environ.get('DATABRICKS_RUNTIME_VERSION',None))

DATABRICKS_RUNTIME_VERSION: client.4.6


Se inicia sesión en DataBricks.

Se selecciona "Compute" en la barra lateral izquierda.

Se hace clic en el botón "Create cluster" para iniciar la configuración.


![](Pantallazo1.png)

## Versión de Databricks Runtime

%md
![](Pantallazo2.png)


![](Pantallazo3.png)


![](Pantallazo5.png)


![](Pantallazo4.png)

Versión de Databricks Runtime: 16.4.x-scala2.12

Tipo de Clúster: Single Node (La versión gratuita no tiene workers).

Núcleos/RAM: 2 Cores / ~16 GB RAM (o valores cercanos dependiendo de la asignación de Azure/AWS por detrás).

Autoscaling: Desactivado.

## Versiones de Python/Spark: spark.version, spark.sparkContext.getConf().getAll().

In [0]:
import sys

print("="*60)
print("🛠️ VERSIONES DE SOFTWARE")
print("="*60)

# 1. Versión de Python
# sys.version da detalles completos (versión mayor.menor.micro)
print(f"🐍 Python Version: {sys.version.split()[0]}")

🛠️ VERSIONES DE SOFTWARE
🐍 Python Version: 3.12.3


In [0]:
# 2. Versión de Spark
# Comando solicitado: spark.version
print(f"✨ Spark Version:  {spark.version}")

print("\n" + "="*60)
print("⚙️ SPARK CONTEXT CONFIGURATIONS (spark.sparkContext.getConf().getAll())")
print("="*60)

✨ Spark Version:  4.0.0

⚙️ SPARK CONTEXT CONFIGURATIONS (spark.sparkContext.getConf().getAll())


## Estructura de Almacenamiento


Estructura de almacenamiento (DBFS o Volumes) que utilizarás.

In [0]:
import random

print("="*60)
print("📂 EVIDENCIA DE ALMACENAMIENTO (Vía Unity Catalog/Managed Table)")
print("="*60)

# 1. Definir nombre del esquema (base de datos)
nombre_esquema = "demo_infraestructura"
nombre_tabla = f"{nombre_esquema}.test_storage"

# 2. Crear un Esquema (si no existe)
# Esto usará la ubicación de almacenamiento por defecto asignada a tu usuario/workspace
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {nombre_esquema}")
print(f"✅ Esquema '{nombre_esquema}' verificado.")

# 3. Crear una tabla pequeña de prueba
# Al no especificar 'LOCATION', Databricks asigna automáticamente el almacenamiento seguro (S3/ADLS)
df_prueba = spark.createDataFrame([(1, "Test Data"), (2, "Evidence")], ["id", "dato"])
df_prueba.write.mode("overwrite").saveAsTable(nombre_tabla)
print(f"✅ Tabla '{nombre_tabla}' creada exitosamente.")

# 4. Obtener la ruta física real (Aquí está tu evidencia)
# Usamos DESCRIBE EXTENDED para ver dónde guardó los datos Spark
info_tabla = spark.sql(f"DESCRIBE EXTENDED {nombre_tabla}").filter("col_name = 'Location'").collect()
ruta_fisica = info_tabla[0]['data_type']

print("-" * 60)
print("📍 UBICACIÓN FÍSICA DE LOS DATOS (Evidence):")
print(f"   {ruta_fisica}")
print("-" * 60)
print("Esta ruta confirma que el almacenamiento (Volumes/Managed) está activo y funcional.")

📂 EVIDENCIA DE ALMACENAMIENTO (Vía Unity Catalog/Managed Table)
✅ Esquema 'demo_infraestructura' verificado.
✅ Tabla 'demo_infraestructura.test_storage' creada exitosamente.
------------------------------------------------------------
📍 UBICACIÓN FÍSICA DE LOS DATOS (Evidence):
   
------------------------------------------------------------
Esta ruta confirma que el almacenamiento (Volumes/Managed) está activo y funcional.


## confirmación de la ruta/tabla creada



![](Pantallazo6.png)
![](Pantallazo7.png)

### Obtén datos de Kaggle y crea una tabla


In [0]:
import re

# Sanitize column names: replace spaces with underscores and remove invalid characters
def sanitize_column(col):
    # Replace spaces with underscores
    col = col.replace(" ", "_")
    # Remove invalid characters: , ; { } ( ) \n \t =
    col = re.sub(r"[,\;{}\(\)\n\t=]", "", col)
    return col

# Apply sanitization to all columns
df_manual = df_manual.toDF(
    *[sanitize_column(col) for col in df_manual.columns]
)

# Save as Delta table
nombre_tabla = "incidentes_manual"
df_manual.write.mode("overwrite").saveAsTable(nombre_tabla)

display(
    spark.sql(f"SELECT * FROM {nombre_tabla} LIMIT 5")
)

DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


In [0]:

# Carga del CSV en un DataFrame aplicando inferencia de esquema
table_name = "incidentes_manual"
csv_file_path = "/Volumes/workspace/db_vif_colombia/mis_archivos/Reporte_Delito_Violencia_Intrafamiliar_Polic_a_Nacional.csv"

# Cargar el archivo con encabezado y esquema inferido
df = spark.read.csv(
    csv_file_path,
    header=True,       # Incluir la primera fila como encabezado
    inferSchema=True,  # Dejar que Spark determine los tipos de datos
    sep=','
)

# esquema inferido y conteos
print("Esquema Inferido (df.printSchema()):")
df.printSchema()

print(f"\nNúmero de registros cargados (df.count()): {df.count()}")

print("\nMuestra de los primeros 5 registros:")
df.limit(5).display()

Esquema Inferido (df.printSchema()):
root
 |-- DEPARTAMENTO: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- CODIGO DANE: string (nullable = true)
 |-- ARMAS MEDIOS: string (nullable = true)
 |-- FECHA HECHO: string (nullable = true)
 |-- GENERO: string (nullable = true)
 |-- GRUPO ETARIO: string (nullable = true)
 |-- CANTIDAD: integer (nullable = true)


Número de registros cargados (df.count()): 476970

Muestra de los primeros 5 registros:


DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


In [0]:
# Rename DataFrame columns to match the Delta table schema
df_fixed = (
    df.withColumnRenamed("CODIGO DANE", "CODIGO_DANE")
      .withColumnRenamed("ARMAS MEDIOS", "ARMAS_MEDIOS")
      .withColumnRenamed("FECHA HECHO", "FECHA_HECHO")
      .withColumnRenamed("GRUPO ETARIO", "GRUPO_ETARIO")
)

# Overwrite the Delta table with the aligned schema
df_fixed.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"Tabla Delta '{table_name}' creada exitosamente en el catálogo.")

Tabla Delta 'incidentes_manual' creada exitosamente en el catálogo.


In [0]:
%sql
-- Muestra la metadata, columnas, tipos de datos y ubicación física
DESCRIBE EXTENDED incidentes_manual;


col_name,data_type,comment
DEPARTAMENTO,string,null
MUNICIPIO,string,null
CODIGO_DANE,string,null
ARMAS_MEDIOS,string,null
FECHA_HECHO,string,null
GENERO,string,null
GRUPO_ETARIO,string,null
CANTIDAD,int,null
,,
# Delta Statistics Columns,,


# Validaciones en Spark y SQL


Validación en SQL (DESCRIBE y SHOW CREATE)

In [0]:
%sql
-- A. Descripción Técnica (Columnas y Tipos)
-- Muestra columnas, tipos de dato (int, string, date) y comentarios
DESCRIBE TABLE incidentes_manual;

col_name,data_type,comment
DEPARTAMENTO,string,null
MUNICIPIO,string,null
CODIGO_DANE,string,null
ARMAS_MEDIOS,string,null
FECHA_HECHO,string,null
GENERO,string,null
GRUPO_ETARIO,string,null
CANTIDAD,int,null


Validación en PySpark (printSchema)

In [0]:
print("="*60)
print("🐍 VALIDACIÓN DE METADATOS (PySpark)")
print("="*60)

# 1. Cargamos la tabla desde el catálogo para asegurar que leemos lo persistido
df_validacion = spark.table("incidentes_manual")

# 2. Imprimimos el esquema (Árbol de estructura)
print(f"Tabla: incidente_manual")
df_validacion.printSchema()

🐍 VALIDACIÓN DE METADATOS (PySpark)
Tabla: incidente_manual
root
 |-- DEPARTAMENTO: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- CODIGO_DANE: string (nullable = true)
 |-- ARMAS_MEDIOS: string (nullable = true)
 |-- FECHA_HECHO: string (nullable = true)
 |-- GENERO: string (nullable = true)
 |-- GRUPO_ETARIO: string (nullable = true)
 |-- CANTIDAD: integer (nullable = true)



Opción PySpark: df.describe() .show()

In [0]:
print("="*60)
print("📊 ESTADÍSTICAS DESCRIPTIVAS (PySpark)")
print("="*60)

# 1. Cargamos la tabla
df_stats = spark.table("incidentes_manual")

# 2. Ejecutamos describe() sobre la métrica principal
# show() muestra la tabla en formato texto
df_stats.select("CANTIDAD").describe().show()

# Si quieres ver estadísticas de fechas (Min/Max), se hace aparte para mayor claridad:
print("Rango de fechas:")
df_stats.select("FECHA_HECHO").describe().show()

📊 ESTADÍSTICAS DESCRIPTIVAS (PySpark)
+-------+------------------+
|summary|          CANTIDAD|
+-------+------------------+
|  count|            476970|
|   mean|1.7077635910015305|
| stddev|3.3386471151633903|
|    min|                 1|
|    max|               130|
+-------+------------------+

Rango de fechas:
+-------+-----------------+
|summary|      FECHA_HECHO|
+-------+-----------------+
|  count|           476970|
|   mean|          44277.5|
| stddev|47.21682508840013|
|    min|        1/01/2010|
|    max|        9/12/2020|
+-------+-----------------+



Opción SQL: Funciones Agregadas

In [0]:
%sql
-- Resumen de Calidad de Datos y Métricas
SELECT 
    -- 1. Volumetría
    COUNT(*) as total_registros,
    
    -- 2. Coherencia Temporal (Validar que las fechas sean reales)
    MIN(fecha_hecho) as fecha_mas_antigua,
    MAX(fecha_hecho) as fecha_mas_reciente,
    
    -- 3. Estadísticas de la Métrica 'cantidad'
    SUM(cantidad) as total_incidentes_acumulados,
    ROUND(AVG(cantidad), 2) as promedio_por_hecho,
    MAX(cantidad) as pico_maximo_incidentes,
    
    -- 4. Control de Calidad (¿Hay nulos?)
    SUM(CASE WHEN cantidad IS NULL THEN 1 ELSE 0 END) as conteo_nulos_cantidad
FROM incidentes_manual;

total_registros,fecha_mas_antigua,fecha_mas_reciente,total_incidentes_acumulados,promedio_por_hecho,pico_maximo_incidentes,conteo_nulos_cantidad
476970,1/01/2010,9/12/2020,814552,1.71,130,0


Consultas SELECT y GROUP BY: equivalentes en Spark y SQL comparando resultados.

Versión PySpark (DataFrame API)

In [0]:
from pyspark.sql.functions import col, sum, desc

print("="*60)
print("🐍 1. RESULTADO EN PYSPARK (Agrupación por Municipio)")
print("="*60)

# 1. Leemos la tabla persistida
df_query = spark.table("incidentes_manual")

# 2. Construimos la consulta:
#    SELECT municipio, SUM(cantidad) FROM ... GROUP BY municipio ORDER BY total DESC LIMIT 5
resultado_pyspark = df_query.groupBy("MUNICIPIO") \
    .agg(sum("CANTIDAD").alias("TOTAL_INCIDENTES")) \
    .orderBy(desc("TOTAL_INCIDENTES")) \
    .limit(5)

# 3. Mostramos el resultado
resultado_pyspark.show()

🐍 1. RESULTADO EN PYSPARK (Agrupación por Municipio)
+------------------+----------------+
|         MUNICIPIO|TOTAL_INCIDENTES|
+------------------+----------------+
|  BOGOTÁ D.C. (CT)|          212649|
|     MEDELLÍN (CT)|           56399|
|         CALI (CT)|           48533|
|  BUCARAMANGA (CT)|           19012|
|VILLAVICENCIO (CT)|           16999|
+------------------+----------------+



MIN(fecha_hecho) / MAX: Permite detectar fechas basura (ej. año 1900 o 2200).

conteo_nulos_cantidad: Debería ser 0. Si es mayor, significa que la carga falló en interpretar algunos números y los dejó vacíos.

Versión SQL (Standard SQL)

In [0]:
%sql
-- ============================================================
-- 🗄️ 2. RESULTADO EN SQL (Equivalente exacto)
-- ============================================================

SELECT 
    MUNICIPIO, 
    SUM(cantidad) as TOTAL_INCIDENTES
FROM incidentes_manual
GROUP BY MUNICIPIO
ORDER BY total_incidentes DESC
LIMIT 5;

MUNICIPIO,TOTAL_INCIDENTES
BOGOTÁ D.C. (CT),212649
MEDELLÍN (CT),56399
CALI (CT),48533
BUCARAMANGA (CT),19012
VILLAVICENCIO (CT),16999


Consistencia: Compara la columna total_incidentes de la celda Python con la de la celda SQL. Los números deben ser idénticos.

Ordenamiento: Ambos listados deben mostrar los municipios en el mismo orden (de mayor a menor).

Conclusión: Esto valida que la tabla incidente está correctamente indexada y accesible tanto para el motor de computación (Spark Core) como para el motor de consulta (Spark SQL).

Conteos y muestras_Validación en SQL

In [0]:
%sql
-- D. CONTEO AGRUPADO SIMPLE
-- ¿Cuántos registros hay por género? (Validación de categorías)
SELECT genero, count(*) as total 
FROM incidentes_manual
GROUP BY genero;

genero,total
MASCULINO,109759
FEMENINO,366907
NO REPORTA,294
-,7
null,3


In [0]:
%sql
-- A. CONTEO TOTAL (Validación de Volumetría)
-- Verifica que el número coincida con las filas de tu CSV original.
SELECT count(*) as total_registros FROM incidentes_manual;

total_registros
476970


In [0]:
%sql
-- B. MUESTRA LIMITADA (LIMIT)
-- Trae solo 5 filas para verificar visualmente que las columnas no están desplazadas.
SELECT * FROM incidentes_manual LIMIT 5;

DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


In [0]:
%sql
-- C. FILTRO POR CAMPO (Validación de Lógica)
-- Ejemplo: Verificar incidentes donde la cantidad reportada sea mayor a 0 (filtrar registros vacíos/ceros)
SELECT * FROM incidentes_manual
WHERE cantidad > 0 
LIMIT 5;

DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


Validación en PySpark

In [0]:
from pyspark.sql.functions import col

print("="*60)
print("🐍 VALIDACIONES DE FILTROS Y CONTEOS (PySpark)")
print("="*60)

# 1. Cargar Tabla
df_val = spark.table("incidentes_manual")

# 2. CONTEO TOTAL (.count)
total_filas = df_val.count()
print(f"🔹 Total de registros en el DataFrame: {total_filas}")

# 3. FILTRO Y MUESTRA (.filter + .show)
# Equivalente a: WHERE cantidad >= 1 LIMIT 5
print("\n🔹 Muestra de 5 registros con Cantidad >= 1:")
df_val.filter(col("cantidad") >= 1).show(5)

# 4. FILTRO ESPECÍFICO
# Validamos si podemos buscar texto específico (ej. un municipio o género)
# Nota: Ajusta 'Masculino' si tu dataset usa otros términos (ej. 'HOMBRE')
filtro_genero = "Masculino" 
conteo_filtrado = df_val.filter(col("genero") == filtro_genero).count()

print(f"\n🔹 Validación de filtro por campo 'genero':")
print(f"   Se encontraron {conteo_filtrado} registros para '{filtro_genero}'")

🐍 VALIDACIONES DE FILTROS Y CONTEOS (PySpark)
🔹 Total de registros en el DataFrame: 476970

🔹 Muestra de 5 registros con Cantidad >= 1:
+------------+-----------------+-----------+--------------------+-----------+---------+------------+--------+
|DEPARTAMENTO|        MUNICIPIO|CODIGO_DANE|        ARMAS_MEDIOS|FECHA_HECHO|   GENERO|GRUPO_ETARIO|CANTIDAD|
+------------+-----------------+-----------+--------------------+-----------+---------+------------+--------+
|   ATLÁNTICO|BARRANQUILLA (CT)|    8001000|ARMA BLANCA / COR...|  1/01/2010|MASCULINO|     ADULTOS|       1|
|      BOYACÁ|          DUITAMA|   15238000|ARMA BLANCA / COR...|  1/01/2010| FEMENINO|     ADULTOS|       1|
|     CAQUETÁ|      PUERTO RICO|   18592000|ARMA BLANCA / COR...|  1/01/2010|MASCULINO|     ADULTOS|       1|
|    CASANARE|             MANÍ|   85139000|ARMA BLANCA / COR...|  1/01/2010| FEMENINO|     ADULTOS|       1|
|CUNDINAMARCA| BOGOTÁ D.C. (CT)|   11001000|ARMA BLANCA / COR...|  1/01/2010| FEMENINO|     AD

COUNT(*) vs .count(): Ambos números coinciden, lo que valida que no hubo pérdida de datos al leer la tabla en diferentes lenguajes.

LIMIT / .show(): Confirma que los datos son legibles y que las columnas (municipio, fecha, cantidad) contienen el tipo de dato correcto (no hay texto en la columna de fechas, por ejemplo).

Filtros (WHERE): Demuestra que la tabla está lista para responder preguntas de negocio (ej. "¿Cuántos hombres fueron afectados?").

## Ventajas y desventajas: SQL vs Spark


| Característica | SQL (Lo tradicional) | Spark / PySpark (Big Data) |
| :--- | :--- | :--- |
| **Curva de aprendizaje** | **Baja.** Es muy intuitivo y es el estándar de la industria. Casi que se lee como inglés. | **Media/Alta.** Requiere saber programar (Python/Scala) y entender cómo funciona el procesamiento distribuido. |
| **Flexibilidad** | Limitada a consultas y manejo de datos estructurados. Crear funciones complejas es tedioso. | **Muy Alta.** Puedes mezclar SQL con Python, usar librerías de IA, graficar y limpiar datos en el mismo script. |
| **Rendimiento** | Excelente en bases de datos tradicionales, pero puede sufrir con volúmenes masivos de datos no indexados. | **Escalable.** Está diseñado para procesar Terabytes en memoria y en paralelo. Si un nodo falla, el trabajo sigue. |
| **Integración** | Rey del Business Intelligence (BI) y reportes. Conecta con todo "out of the box". | Ideal para Pipelines de Ingeniería de Datos (ETL) y Ciencia de Datos (Machine Learning). |
| **Depuración (Debug)** | A veces es difícil saber dónde falló una consulta gigante anidada. | Los errores pueden ser crípticos (logs de Java), pero ofrece más control para optimizar la ejecución. |